In [2]:
from keras.layers import Input, Convolution2D, Lambda, merge, Dense, Flatten,MaxPooling2D
from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import SGD
import numpy.random as rng
import numpy as np
import dill as pickle
import matplotlib.pyplot as plt



def W_init(shape,name=None):
    """Initialize weights as in paper"""
    values = rng.normal(loc=0,scale=1e-2,size=shape)
    return K.variable(values,name=name)


input_shape = (105, 105, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convolutional network to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Convolution2D(64,10,10,activation='relu',input_shape=input_shape,init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(128,7,7,activation='relu',init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(128,4,4,activation='relu',init=W_init))
convnet.add(MaxPooling2D())
convnet.add(Convolution2D(256,4,4,activation='relu',init=W_init))
convnet.add(Flatten())
convnet.add(Dense(4096,activation="sigmoid"))
#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#compute the l1 distance between the two encoded digits
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)
optimizer = SGD(0.009,momentum=0.6,nesterov=True,decay=0.0003)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)

Using TensorFlow backend.


In [3]:
with open( "/home/soren/keras-oneshot/train.pickle","r") as f:
    (X,y,c) = pickle.load(f)

with open( "/home/soren/keras-oneshot/val.pickle","r") as f:
    (Xval,yval,cval) = pickle.load(f)
print(X.shape)
a=siamese_net.predict([X[1,1,:,:].reshape(1,105,105,1),X[7,1,:,:].reshape(1,105,105,1) ])
b=siamese_net.predict([X[7,1,:,:].reshape(1,105,105,1),X[1,1,:,:].reshape(1,105,105,1) ])



(964, 20, 105, 105)


In [16]:
class Siamese_Loader:
    """For loading batches and testing tasks to a siamese net"""
    def __init__(self,Xtrain,Xval):
        self.Xval = Xval
        self.Xtrain = Xtrain
        self.n_classes,self.n_examples,self.w,self.h = Xtrain.shape
        self.n_val,self.n_ex_val,_,_ = Xval.shape

    def get_batch(self,n):
        """Create batch of pairs, half same class, half different class"""
        categories = rng.choice(self.n_classes,size=(n,),replace=False)
        pairs=[np.zeros((n, self.h, self.w,1)),np.zeros((n, self.h, self.w,1))]
        targets=np.zeros((n,))
        targets[n//2:] = 1
        for i in range(n):
            category = categories[i]
            idx_1 = rng.randint(0,self.n_examples)
            pairs[0][i,:,:,:] = self.Xtrain[category,idx_1].reshape(self.w,self.h,1)
            idx_2 = rng.randint(0,self.n_examples)
            #pick images of same class for 1st half, different for 2nd
            category_2 = category if i >= n//2 else (category + rng.randint(1,self.n_classes)) % self.n_classes
            pairs[1][i,:,:,:] = self.Xtrain[category_2,idx_2].reshape(self.w,self.h,1)
        return pairs, targets

    def make_oneshot_task(self,N):
        """Create pairs of test image, support set for testing N way one-shot learning. """
        categories = rng.choice(self.n_val,size=(N,),replace=False)
        indices = rng.randint(0,self.n_ex_val,size=(N,))
        true_category = categories[0]
        ex1, ex2 = rng.choice(self.n_examples,replace=False,size=(2,))
        test_image = np.asarray([self.Xval[true_category,ex1,:,:]]*N).reshape(N,self.w,self.h,1)
        support_set = self.Xval[categories,indices,:,:]
        support_set[0,:,:] = self.Xval[true_category,ex2]
        support_set = support_set.reshape(N,self.w,self.h,1)
        pairs = [test_image,support_set]
        targets = np.zeros((N,))
        targets[0] = 1
        return pairs, targets
    
    def test_oneshot(self,model,N,k,verbose=0):
        """Test average N way oneshot learning accuracy of a siamese neural net over n one-shot tasks"""
        pass
        n_correct = 0
        if verbose:
            print("Evaluating {} way task {} times...".format(N,k))
        for i in range(k):
            inputs, targets = self.make_oneshot_task(N)
            probs = model.predict(inputs)
            print(probs)
            print(np.argmax(probs))
            if np.argmax(probs) == 0:
                n_correct+=1
        percent_correct = (100.0*n_correct / k)
        if verbose:
            print("Got {}% {} way one-shot learning accuracy".format(percent_correct,N))
        return percent_correct
    
loader = Siamese_Loader(X,Xval)
b=loader.test_oneshot(siamese_net,N=5,k=3,verbose=True)
print(b)  

Evaluating 5 way task 3 times...
[[ 0.48157385]
 [ 0.50549209]
 [ 0.50719666]
 [ 0.4888424 ]
 [ 0.50703931]]
2
[[ 0.52736533]
 [ 0.52286547]
 [ 0.49043316]
 [ 0.51916617]
 [ 0.51127368]]
0
[[ 0.4806416 ]
 [ 0.49986258]
 [ 0.48568013]
 [ 0.48592973]
 [ 0.48212302]]
1
Got 33.3333333333% 5 way one-shot learning accuracy
33.3333333333


In [ ]:


for i in range(20000):
    (inputs,targets)=get_batch(X,y,64)
    loss=siamese_net.train_on_batch(inputs,targets)
    if i % 170 == 0:
        val_input,val_target=get_batch(Xval,yval,32)
        val_loss=siamese_net.evaluate(val_input,val_target)
        print("iteration: {}, training loss: {}, val loss: {}".format(i,loss, val_loss))
    

(inputs,targets)=get_batch(Xval,yval,64)



plt.matshow(np.hstack([siamese_net.predict(inputs), targets.reshape(64,1)])[16:48],cmap='gray')
plt.show()


In [ ]:
preds=siamese_net.predict(inputs)
#print(np.where(preds==preds.max()))
#print(inputs[0][58].shape)
for idx in range(3):
    print("probability not same:{}".format(preds[idx]))
    plt.matshow(inputs[0][idx].reshape(105,105),cmap='gray')
    plt.matshow(inputs[1][idx].reshape(105,105),cmap='gray')

    plt.show()
#plt.matshow(inputs[1][58])
